public class Version {
  private int versionNumber;
  public Version(int versionNumber) {
     assert versionNumber >= 0;
     this.versionNumber = versionNumber;
  }
  public Version() {
     this(0);
  }
  public int getVersionNumber() {
     return versionNumber;
  }
  public int incrVersionNumber() {
      return versionNumber++;
  }
  public String toString() {
      return "" + versionNumber;
  }
}

Podemos probar en el siguiente código que va incrementando el número de versión varias veces e imprimiéndolo.

In [13]:
Version v = new Version();
System.out.println(v);
v.incrVersionNumber();
System.out.println(v);
v.incrVersionNumber();
System.out.println(v);

0
1
2


Ahora con este código vamos a implementar una bitácora de versiones sobre un fichero. Normalmente la bitácora de versiones lleva un registro de cada versión en la que ha habido cambios en un proyecto. Para ellos vamos a utilizar una lista (`List`) de versiones donde se guarda cada una de las versiones por las que ha pasado nuestro fichero. Nuestro proyecto consta de un solo fichero. El siguiente código genera la bitácora:

In [17]:
List<Version> log = new ArrayList<>();
Version fileVersion = new Version();

Vamos a simular el comportamiento de la bitácora, realizando una secuencia de 5 cambios a través del siguiente procedimiento que simula la ejecución de n versiones.

In [20]:
static void seqChanges(int n) {
    for (int i = 0; i < n; i++) {
        System.out.println("Current version: " + fileVersion.getVersionNumber());
        log.add(fileVersion);
        fileVersion.incrVersionNumber();
        System.out.println("New version: " + fileVersion.getVersionNumber());
    }
}

Probemos que todo (aparentemente) funciona bien:

In [21]:
seqChanges(5);

Current version: 5
New version: 6
Current version: 6
New version: 7
Current version: 7
New version: 8
Current version: 8
New version: 9
Current version: 9
New version: 10


Todo bien, hemos hecho los cambios correspondientes y se han correctamente la versiones `[0,1,2,3,4]`. El siguiente valor a registrar es el 5. Entonces, miremos el siguiente código que debe imprimir lo mismo.

In [22]:
System.out.println(log);

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


In [ ]:
public class Version {
  private int versionNumber;
  public Version(int versionNumber) {
     assert versionNumber >= 0;
     this.versionNumber = versionNumber;
  }
  public Version() {
     this(0);
  }
  public int getVersionNumber() {
     return versionNumber;
  }
  public Version incrVersionNumber() {
      return new Version();
  }
  public String toString() {
      return "" + versionNumber;
  }
}

Ahora el procedimiento seqChanges cambia, en esta caso la función recibe tres valores el número `n` de veces y el valor de la versión actual. Y la función retorna un Pair donde la clave es el valor siguiente de la versión, y el valor es de la bitácora modificada. El siguiente es parte del cuerpo de la función.

In [24]:
class Pair<K,V> {
   private K key;
   private V value;
   Pair(K key, V value) {
      this.key = key;
      this.value = value;
   }
   K getKey() {
      return key;
   }
   V getValue() {
      return value;
   }
}

static Pair<Version,List<Version>> seqChange(int n, Version actual) {
    return new Pair(actual, new ArrayList<>());
}

La función `seqChange` es una versión muy similar a la anterior, no es una **función pura**.

